In [1]:
using JuMP
using CPLEX

In [2]:
model = Model(CPLEX.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: CPLEX

In [3]:
demand = [1500, 2100, 1800, 1950]

4-element Vector{Int64}:
 1500
 2100
 1800
 1950

In [4]:
regular_hrs = 480
max_overtime = 80
production_rate = 160
max_subcontract = 500
regular_pay = 50
overtime_pay = 55
subcontracting_cost = 9000
backlog_cost = 1200
inventory_holding_cost = 50
hiring_cost = 1000
firing_cost = 1200

1200

In [5]:
num_init_workers = 600
inventory_start = 200
inventory_end = 300

300

In [6]:
@variable(model, total_overtime_hrs[1:4], lower_bound = 0)

4-element Vector{VariableRef}:
 total_overtime_hrs[1]
 total_overtime_hrs[2]
 total_overtime_hrs[3]
 total_overtime_hrs[4]

In [7]:
@variable(model, w[1:4], lower_bound = 0, Int)

4-element Vector{VariableRef}:
 w[1]
 w[2]
 w[3]
 w[4]

In [8]:
@variable(model, w_h[1:4], lower_bound = 0, Int)

4-element Vector{VariableRef}:
 w_h[1]
 w_h[2]
 w_h[3]
 w_h[4]

In [9]:
@variable(model, w_f[1:4], lower_bound = 0, Int)

4-element Vector{VariableRef}:
 w_f[1]
 w_f[2]
 w_f[3]
 w_f[4]

In [10]:
@variable(model, production[1:4], lower_bound = 0, Int)

4-element Vector{VariableRef}:
 production[1]
 production[2]
 production[3]
 production[4]

In [11]:
@variable(model, subcontract[1:4], lower_bound = 0, Int)

4-element Vector{VariableRef}:
 subcontract[1]
 subcontract[2]
 subcontract[3]
 subcontract[4]

In [12]:
@variable(model, inventory[1:4], lower_bound = 0, Int)

4-element Vector{VariableRef}:
 inventory[1]
 inventory[2]
 inventory[3]
 inventory[4]

In [13]:
@variable(model, backlog[1:4], lower_bound = 0, Int)

4-element Vector{VariableRef}:
 backlog[1]
 backlog[2]
 backlog[3]
 backlog[4]

In [14]:
@constraint(model, w[1]== num_init_workers + w_h[1] - w_f[1])

w[1] - w_h[1] - w_f[1] == 600

In [15]:
@constraint(model, production[1] + inventory_start + subcontract[1] == demand[1] + inventory[1] + backlog[1])

production[1] + subcontract[1] - inventory[1] - backlog[1] == 1300

In [16]:
@constraint(model, w[2:4]== w[1:3] + w_h[2:4] - w_f[2:4])

[-w[1] + w[2] - w_h[2] - w_f[2], -w[2] + w[3] - w_h[3] - w_f[3], -w[3] + w[4] - w_h[4] - w_f[4]] in MathOptInterface.Zeros(3)

In [17]:
@constraint(model,production[2:4] + inventory[1:3] + subcontract[2:4] == demand[2:4] + inventory[2:4] + backlog[1:3] + backlog[2:4])

[production[2] + subcontract[2] + inventory[1] - inventory[2] - backlog[1] - backlog[2] - 2100, production[3] + subcontract[3] + inventory[2] - inventory[3] - backlog[2] - backlog[3] - 1800, production[4] + subcontract[4] + inventory[3] - inventory[4] - backlog[3] - backlog[4] - 1950] in MathOptInterface.Zeros(3)

In [18]:
@constraint(model,production[2:4] + inventory[1:3] + subcontract[2:4] == demand[2:4] + inventory[2:4] + backlog[1:3] + backlog[2:4])

[production[2] + subcontract[2] + inventory[1] - inventory[2] - backlog[1] - backlog[2] - 2100, production[3] + subcontract[3] + inventory[2] - inventory[3] - backlog[2] - backlog[3] - 1800, production[4] + subcontract[4] + inventory[3] - inventory[4] - backlog[3] - backlog[4] - 1950] in MathOptInterface.Zeros(3)

In [19]:
@constraint(model, total_overtime_hrs <= max_overtime * w[1:4])

[total_overtime_hrs[1] - 80 w[1], total_overtime_hrs[2] - 80 w[2], total_overtime_hrs[3] - 80 w[3], total_overtime_hrs[4] - 80 w[4]] in MathOptInterface.Nonpositives(4)

In [20]:
@constraint(model, production <= w[1:4] * regular_hrs / production_rate + total_overtime_hrs / production_rate)

[-0.00625 total_overtime_hrs[1] - 3 w[1] + production[1], -0.00625 total_overtime_hrs[2] - 3 w[2] + production[2], -0.00625 total_overtime_hrs[3] - 3 w[3] + production[3], -0.00625 total_overtime_hrs[4] - 3 w[4] + production[4]] in MathOptInterface.Nonpositives(4)

In [21]:
@constraint(model, subcontract[1:4] .<= max_subcontract)

4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 subcontract[1] <= 500
 subcontract[2] <= 500
 subcontract[3] <= 500
 subcontract[4] <= 500

In [22]:

@constraint(model, inventory[4] == inventory_end)

inventory[4] == 300

In [23]:
total_cost = sum(w .* regular_hrs .* regular_pay) + sum(hiring_cost .* w_h) + sum(firing_cost .* w_f) + sum(inventory_holding_cost .* inventory) + sum(backlog_cost .* backlog) + sum(subcontracting_cost .* subcontract) + sum(overtime_pay .* total_overtime_hrs)

24000 w[1] + 24000 w[2] + 24000 w[3] + 24000 w[4] + 1000 w_h[1] + 1000 w_h[2] + 1000 w_h[3] + 1000 w_h[4] + 1200 w_f[1] + 1200 w_f[2] + 1200 w_f[3] + 1200 w_f[4] + 50 inventory[1] + 50 inventory[2] + 50 inventory[3] + 50 inventory[4] + 1200 backlog[1] + 1200 backlog[2] + 1200 backlog[3] + 1200 backlog[4] + 9000 subcontract[1] + 9000 subcontract[2] + 9000 subcontract[3] + 9000 subcontract[4] + 55 total_overtime_hrs[1] + 55 total_overtime_hrs[2] + 55 total_overtime_hrs[3] + 55 total_overtime_hrs[4]

In [24]:
@objective(model, Min, total_cost)

24000 w[1] + 24000 w[2] + 24000 w[3] + 24000 w[4] + 1000 w_h[1] + 1000 w_h[2] + 1000 w_h[3] + 1000 w_h[4] + 1200 w_f[1] + 1200 w_f[2] + 1200 w_f[3] + 1200 w_f[4] + 50 inventory[1] + 50 inventory[2] + 50 inventory[3] + 50 inventory[4] + 1200 backlog[1] + 1200 backlog[2] + 1200 backlog[3] + 1200 backlog[4] + 9000 subcontract[1] + 9000 subcontract[2] + 9000 subcontract[3] + 9000 subcontract[4] + 55 total_overtime_hrs[1] + 55 total_overtime_hrs[2] + 55 total_overtime_hrs[3] + 55 total_overtime_hrs[4]

In [25]:
optimize!(model)

Version identifier: 22.1.1.0 | 2022-11-26 | 9160aff4d
Tried aggregator 2 times.
MIP Presolve eliminated 8 rows and 5 columns.
MIP Presolve added 1 rows and 1 columns.
Aggregator did 2 substitutions.
Reduced MIP has 15 rows, 26 columns, and 51 nonzeros.
Reduced MIP has 0 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Found incumbent of value 7.3939200e+07 after 0.02 sec. (0.10 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 1 rows and 1 columns.
MIP Presolve added 1 rows and 1 columns.
Reduced MIP has 15 rows, 26 columns, and 51 nonzeros.
Reduced MIP has 0 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objec

In [26]:
@show value.(w)

value.(w) = [620.0, 621.0, 621.0, 621.0]


4-element Vector{Float64}:
 620.0
 621.0
 621.0
 621.0

In [27]:
@show value.(inventory)

value.(inventory) = [560.0, 323.0, 387.0, 300.0]


4-element Vector{Float64}:
 560.0
 323.0
 387.0
 300.0

In [28]:
@show value.(backlog)

value.(backlog) = [0.0, 0.0, 0.0, 0.0]


4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

In [29]:
@show value.(w_h)

value.(w_h) = [20.0, 1.0, 0.0, 0.0]


4-element Vector{Float64}:
 20.0
  1.0
  0.0
  0.0

In [30]:
@show value.(w_f)

value.(w_f) = [0.0, 0.0, 0.0, 0.0]


4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

In [31]:
@show value.(subcontract)

value.(subcontract) = [0.0, 0.0, 1.0, 0.0]


4-element Vector{Float64}:
 0.0
 0.0
 1.0
 0.0

In [32]:
@show value.(production)

value.(production) = [1860.0, 1863.0, 1863.0, 1863.0]


4-element Vector{Float64}:
 1860.0
 1863.0
 1863.0
 1863.0

In [33]:
@show value.(total_overtime_hrs)

value.(total_overtime_hrs) = [0.0, 0.0, 0.0, 0.0]


4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

In [34]:
@show objective_value(model)

objective_value(model) = 5.97005e7


5.97005e7